In [1]:
import warnings
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns
import pandas
import numpy as np
from scipy.stats import norm
from statsmodels.tsa.ar_model import AutoReg

%matplotlib inline
%config InlineBackend.figure_format = "retina"

sns.set(rc={"figure.figsize" : (25, 15)})
sns.set(font_scale=2)
sns.set_style("ticks")

warnings.simplefilter("ignore", FutureWarning)  # ignore some seaborn warnings until they update numpy

In [2]:
df = pandas.read_csv("NHL_API_point_totals_by_team_season_raw_1996-2003.csv")
vegas_df = df[df["team"] == "VGK"].copy()
vegas_df

,team,season_end,total_points,points_rank
673,VGK,2018,109,5
704,VGK,2019,93,15
735,VGK,2020,86,7
766,VGK,2021,82,2
797,VGK,2022,94,17
829,VGK,2023,111,5


In [3]:
vegas_df["season_end"] = vegas_df["season_end"].apply(lambda x : date(month=5, day=1, year=x))
vegas_df["season_end"] = pandas.to_datetime(vegas_df["season_end"])

In [4]:
vegas_df = vegas_df.set_index("season_end")

In [125]:
ar = AutoReg(vegas_df["total_points"][0:-1], lags=1, trend="ct").fit()

/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-MAY will be used.
  self._init_dates(dates, freq)


In [126]:
ar.forecast(1)

/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


2023-05-01    121.511905
Freq: AS-MAY, dtype: float64

In [127]:
ar.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:           total_points   No. Observations:                    5
Model:                     AutoReg(1)   Log Likelihood                  -9.268
Method:               Conditional MLE   S.D. of innovations              2.455
Date:                Mon, 02 Oct 2023   AIC                             26.536
Time:                        14:22:26   BIC                             24.082
Sample:                    05-01-2019   HQIC                            21.149
                         - 05-01-2022                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             -84.1071     49.398     -1.703      0.089    -180.926      12.712
trend              12.2619      3.681      3.331      0.001       5.047      19.477
total_points.L1     1.4048      0.399      3.518      0.000       0.622       2.187
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            0.7119           +0.0000j            0.7119            0.0000
-----------------------------------------------------------------------------
"""

In [128]:
ar = AutoReg(vegas_df["total_points"], lags=1, trend="ct").fit()

/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-MAY will be used.
  self._init_dates(dates, freq)


In [129]:
ar.forecast(1)

/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


2024-05-01    137.595593
Freq: AS-MAY, dtype: float64

In [130]:
resid_std = ar.resid.std()

In [131]:
resid_std

2.76430111409753

In [132]:
ar.forecast(1) - (1.96 * resid_std)

/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


2024-05-01    132.177563
Freq: AS-MAY, dtype: float64

In [133]:
ar.forecast(1) + (1.96 * resid_std)

/Users/diane/miniconda3/lib/python3.11/site-packages/statsmodels/tsa/deterministic.py:302: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)


2024-05-01    143.013624
Freq: AS-MAY, dtype: float64